In [162]:
import regex as re
import pandas as pd
from textblob import TextBlob
from emoji_translate.emoji_translate import Translator
from typing import Tuple
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [163]:
def preprocess_tweet(tweet):
    """ Function used for a first preprocessing step. """
    hashtag = re.compile(r"^#\S+|\s#\S+")
    at_someone = re.compile(r"^@\S+|\s@\S+")
    url = re.compile(r"https?://\S+")
    tweet_without_hashtag = hashtag.sub(' ', tweet)
    tweet_without_at_and_hashtag = at_someone.sub(' person', tweet_without_hashtag)
    cleaned_text = url.sub(" fan", tweet_without_at_and_hashtag)

    cleaned_text_lower = cleaned_text.strip().lower()
    cleaned_text_lower_splitted = cleaned_text_lower.split()
    if cleaned_text_lower_splitted == "rt":
        cleaned_text_lower = " ".join(cleaned_text_lower_splitted[1:])

    emo = Translator(exact_match_only=False)
    cleaned_text_lower_emojiless = emo.demojify(cleaned_text_lower)

    clean_text = TextBlob(cleaned_text_lower_emojiless).correct()
    return str(clean_text)

def get_tweet_sentiment(tweet):
    # create TextBlob object of tweet text
    analysis = TextBlob(tweet)
    return analysis.polarity

def print_return_sentiments(df_tweet_column: pd.Series) -> Tuple:
    cleaned_tweets = []
    sentiment_lst = []
    for tweet in df_tweet_column:
        cleaned_tweet = preprocess_tweet(tweet)
        cleaned_tweets.append(cleaned_tweet)
        sentiment = get_tweet_sentiment(cleaned_tweet)

        print(tweet)
        if sentiment > 0:
            sentiment_lst.append("Positive")
            print("Positive tweet")
        elif sentiment < 0:
            sentiment_lst.append("Negative")
            print("Negative tweet")
        else:
            sentiment_lst.append("Neutral")
            print("Neutral tweet")
    return sentiment_lst, cleaned_tweets

def show_sentiment_distribution(df_sentiment_column: pd.Series):
    sentiment_df = df_sentiment_column.value_counts().to_frame().reset_index()
    sentiment_df.columns = ["sentiment","count"]
    fig = px.pie(sentiment_df, names="sentiment", values="count",
                 hole=.3)
    fig.show()

In [164]:
df = pd.read_csv("files/brooklyn99.csv",parse_dates=["created_at"])

In [165]:
df["sentiment"], df["cleaned_tweet"] = \
    print_return_sentiments(df["text"])

@nbcbrooklyn99 All of these are wrong unless they end in “I love you”
Neutral tweet
RT @9drink_amy: @nbcbrooklyn99 How to find the best ‘stume for Halloween without everyone laughing at you?
Positive tweet
@nbcbrooklyn99 "How to retrieve my sperms from the sperm bank without letting my ex wife know"
Neutral tweet
@CorridorDigital quite funny how @nbcbrooklyn99 left a green glove to be rotoscoped, so much more work than just ha… https://t.co/qd4rSAHEhq
Positive tweet
@nbcbrooklyn99 Mervyn's clothing store near me
Positive tweet
where can i buy a jake peralta? i am ready to sell my kidney @nbcbrooklyn99
Positive tweet
@nbcbrooklyn99 How to make your best friend not have any other friends/best friend?
Positive tweet
WAIT! @EugeneLeeYang was in @nbcbrooklyn99??? How did I not know that? https://t.co/lQCK02nh6y
Neutral tweet
@nbcbrooklyn99 Jake paralta  Charles Boyle Best Friend
Positive tweet
@melsperalta @nbcbrooklyn99 @melissafumero Omg thank you 😭❤️
Neutral tweet
@hedair01 @nbcbrooklyn9

In [166]:
show_sentiment_distribution(df["sentiment"])

In [192]:
df_date_index = df.sort_values('created_at').set_index('created_at')
df_date_index = df_date_index.resample("D").value.count()

AttributeError: 'DatetimeIndexResampler' object has no attribute 'value'

In [193]:
print(df_date_index)

                                  username  \
created_at                                   
2021-10-03 14:33:45+00:00     DanielleSATM   
2021-10-03 15:40:35+00:00           Wolven   
2021-10-03 16:03:55+00:00          dwtddl_   
2021-10-03 16:18:24+00:00  lovelyeditsssss   
2021-10-03 17:58:12+00:00     DanielleSATM   
...                                    ...   
2021-10-13 12:06:22+00:00       Fargoesque   
2021-10-13 12:12:00+00:00       9drink_amy   
2021-10-13 12:24:21+00:00          kara_jn   
2021-10-13 12:28:10+00:00     aditibadwaya   
2021-10-13 12:28:58+00:00   HeavySweetness   

                                                                        text  \
created_at                                                                     
2021-10-03 14:33:45+00:00  Just waking up? I've got pieces on #Batwoman a...   
2021-10-03 15:40:35+00:00  The final season of #BrooklynNineNine dealt wi...   
2021-10-03 16:03:55+00:00                        goodbye 😭 #BrooklynNineNine   
2